<a href="https://colab.research.google.com/github/Alenushka2013/Projects/blob/main/Deloitte_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестове завдання

*Опис:*
 - на закладці Input_port відображені дані по частці forward-міграцій умовного кредитного портфелю
 - на закладці Input_macro відображені дані по макроекономічним показникам
 - на закладці Macro_definition відображена розшифровка кодів макро-індексів

*Завдання:*
 - Створення моделі, яка відображає чуттєвість реакції умовного портфеля до макроекономічних показників або опис алгоритму створення такої моделі

*Очікуваний результат:*
 На закладці Results приведений приклад (сіра заливка) очікуваного рішення завдання:
 - опис обраного методу моделювання
 - перелік факторів, які були включені в кінцеву (фінальну) модель
 - опис кінцевої формули та основних параметрів моделі
 - результати бек-тестування моделі.

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
# Завантаження даних
file_path = '/content/Modeler_test_basic_ukr_v2.xlsx'  # Шлях до файлу з  даними
input_port = pd.read_excel(file_path, sheet_name='Input_port')
input_macro = pd.read_excel(file_path, sheet_name='Input macro')


In [3]:
# Об'єднання даних по даті
data = pd.merge(input_port, input_macro, on="date")


Оскільки макроекономічні дані починаються на рік раніше, ці додаткові спостереження можуть містити важливу інформацію для моделі, враховуючи, що ці показники часто впливають із запізненням.

Додамо до моделі лагові значення макроекономічних показників, тобто значення  за попередні місяці для кожного значення частки forward-міграцій. Наприклад, можна додати фактори з лагом в 1, 3, або більше місяців, щоб виявити потенційний ефект попередніх значень.

In [4]:
# Створення лагів для всіх макроекономічних факторів
lags = [1, 3]
for lag in lags:
    for col in input_macro.columns[1:]:  # Пропускаємо колонку "date"
        data[f"{col}_lag_{lag}"] = data[col].shift(lag)


In [5]:
# Видалення рядків з пропущеними значеннями після створення лагів
data.dropna(inplace=True)


In [6]:
# Вибір цільової змінної та факторів для моделі
X = data.drop(columns=["date", "Portfolio forward migration rate"])  # Виключення дати та цільової змінної
y = data["Portfolio forward migration rate"]


In [7]:
X.shape

(14, 51)

Такий підхід значно в перстпективі (якщо даних буде значно більше) збільшує   обсяг обчислень, тому варто обрати найбільш значущі фактори, за якими потім і буде створюватися модель.

Лишається всього 14  значень для цільової змінної, тому викорстовувати підходи які потребують розбиття на навчальну і тестову вибірки не доцільно.

Визначення важливості факторів за допомогою випадкового лісу може бути виконано на всьому наборі даних, не розбиваючи його. Це дозволить  зрозуміти, які фактори найбільше впливають на цільову змінну.

In [8]:
# Використання випадкового лісу для оцінки важливості факторів на всіх даних
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)


RandomForestRegressor(random_state=42)

In [9]:
# Оцінка важливості факторів
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)
top_factors = feature_importances.nlargest(5)  # Вибір 5 найбільш важливих факторів
print("Top important factors:", top_factors.index.tolist())


Top important factors: ['CPI_MY', 'CCPI_MY', 'Trade_MY_lag_1', 'BBI_MY_lag_3', 'Industr_MY_lag_1']


In [10]:
# Побудова нової моделі з найважливішими факторами
X_selected = X[top_factors.index]  # Вибір лише значущих факторів


Модель SARIMAX була обрана через її здатність враховувати сезонні ефекти, використання екзогенних змінних та можливість точного прогнозування навіть на малих вибірках даних.

In [11]:
# Створення моделі
reduced_model = SARIMAX(y, exog=X_selected, order=(1, 0, 1))  # Використовуємо SARIMAX для обробки лагів
reduced_model_fit = reduced_model.fit(disp=False)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
# Прогнозування на всьому наборі даних
y_pred_reduced = reduced_model_fit.predict(start=0, end=len(y)-1, exog=X_selected)


In [13]:
# Оцінка нової моделі
mse_reduced = mean_squared_error(y, y_pred_reduced)
print(f"Mean Squared Error (MSE) of the reduced model: {mse_reduced}")


Mean Squared Error (MSE) of the reduced model: 5.400746414645718e-06


In [14]:
# Виведення параметрів моделі
print(reduced_model_fit.summary())

                                      SARIMAX Results                                       
Dep. Variable:     Portfolio forward migration rate   No. Observations:                   14
Model:                             SARIMAX(1, 0, 1)   Log Likelihood                  65.039
Date:                              Tue, 12 Nov 2024   AIC                           -114.078
Time:                                      13:50:29   BIC                           -108.965
Sample:                                           0   HQIC                          -114.551
                                               - 14                                         
Covariance Type:                                opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
CPI_MY              -0.0159      0.044     -0.359      0.720      -0.103       0.071
C